<a href="https://colab.research.google.com/github/adidwiribowo/Implementasi-Arsitektur-EfficientNetV2-S-untuk-Klasifikasi-Citra-Rempah/blob/main/EfficienNetV2_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pip Install**

In [ ]:
!pip install gradio

In [ ]:
!pip install --upgrade jax jaxlib

# **Klasifikasi**

In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from collections import Counter
from tqdm import tqdm
import gradio as gr

In [ ]:
# Set Path Dataset
dataset_mentah = '/content/drive/MyDrive/Klasifikasi Rempah/New Dataset Rempah'
dataset_baru = '/content/drive/MyDrive/Klasifikasi Rempah/EfficientNetV2/Dataset_Split'
train_dataset = os.path.join(dataset_baru, 'train')
val_dataset = os.path.join(dataset_baru, 'validation')
test_dataset = '/content/drive/MyDrive/Klasifikasi Rempah/EfficientNetV2/test'

In [ ]:
# Fungsi untuk menghitung jumlah gambar dalam setiap kelas
def count_images_per_class(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            num_images = len([img for img in os.listdir(class_path) if img.endswith(('.JPG', '.jpg', '.PNG', '.png'))])
            class_counts[class_name] = num_images
    return class_counts

# Menampilkan jumlah gambar dalam setiap kelas sebelum split
print("Jumlah gambar per kelas SEBELUM split:")
before_split_counts = count_images_per_class(dataset_mentah)
for class_name, count in before_split_counts.items():
    print(f"{class_name}: {count} gambar")

# Stratified Dataset Split
def stratified_split(dataset_mentah, train_dataset, val_dataset, val_size=0.3, random_seed=42):
    for class_name in os.listdir(dataset_mentah):
        class_path = os.path.join(dataset_mentah, class_name)
        if os.path.isdir(class_path):
            images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.JPG', '.jpg', '.PNG', '.png'))]
            if not images:
                print(f"Peringatan: Tidak ada gambar di folder {class_name}")
                continue

            # Stratified Splitting
            train_images, val_images = train_test_split(
                images, test_size=val_size, stratify=[class_name] * len(images), random_state=random_seed
            )

            # Copy Images ke folder train dan validation
            for img_list, dst_dir in zip([train_images, val_images], [train_dataset, val_dataset]):
                dst_class_dir = os.path.join(dst_dir, class_name)
                os.makedirs(dst_class_dir, exist_ok=True)
                for img_path in img_list:
                    shutil.copy(img_path, dst_class_dir)

# Memanggil fungsi untuk membagi data
stratified_split(dataset_mentah, train_dataset, val_dataset)
print("Splitting Dataset Selesai!")

# Menampilkan jumlah gambar dalam setiap kelas setelah split
print("\nJumlah gambar per kelas SETELAH split:")
print("\nTraining Set:")
train_counts = count_images_per_class(train_dataset)
for class_name, count in train_counts.items():
    print(f"{class_name}: {count} gambar")

print("\nValidation Set:")
val_counts = count_images_per_class(val_dataset)
for class_name, count in val_counts.items():
    print(f"{class_name}: {count} gambar")


In [ ]:
# Fungsi untuk menghitung jumlah gambar dalam setiap kelas
def count_images_per_class(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            num_images = len([img for img in os.listdir(class_path) if img.endswith(('.JPG', '.jpg', '.PNG', '.png'))])
            class_counts[class_name] = num_images
    return class_counts

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.6, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Flow Data untuk Training, Validation, dan Test
train_generator = train_datagen.flow_from_directory(
    train_dataset, target_size=(240, 240), batch_size=16, class_mode='categorical')
validation_generator = val_test_datagen.flow_from_directory(
    val_dataset, target_size=(240, 240), batch_size=16, class_mode='categorical')
test_generator = val_test_datagen.flow_from_directory(
    test_dataset, target_size=(240, 240), batch_size=16, class_mode='categorical')

In [ ]:
# Menampilkan beberapa gambar hasil augmentasi
def show_augmented_images(generator, batch_size=10):
    class_labels = list(generator.class_indices.keys())
    images, labels = next(generator)
    plt.figure(figsize=(12, 12))
    for i in range(batch_size):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i])
        plt.title(f"Class: {class_labels[np.argmax(labels[i])]}")
        plt.axis("off")
    plt.show()

show_augmented_images(train_generator)

In [ ]:
# Definisikan Model EfficientNetV2S dengan Fine-Tuning
def build_model(num_classes):
    base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(240, 240, 3))
    for layer in base_model.layers[:70]:
        layer.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.6),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
    model.summary()
    return model

In [ ]:
# Training Model
num_classes = len(train_generator.class_indices)
model = build_model(num_classes)

callbacks = [
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    TensorBoard(log_dir='./logs', histogram_freq=1)
]


history = model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=callbacks)

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

plot_training_history(history)

In [ ]:
# Evaluasi Model Menggunakan Data Uji
results = model.evaluate(test_generator)
print(f"Test Loss: {results[0]:.4f}, Test Accuracy: {results[1]:.4f}, Top-3 Accuracy: {results[2]:.4f}")

In [ ]:
# Prediksi pada data uji
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())


# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
# Visualisasi Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


# Classification Report (Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

In [ ]:
# Fungsi prediksi gambar
def predict_image(image):
    # Preprocessing gambar
    img = load_img(image, target_size=(240, 240))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi dengan model
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions)
    confidence = predictions[0][predicted_class_idx]
    predicted_class = list(train_generator.class_indices.keys())[predicted_class_idx]

    # Menentukan output berdasarkan confidence
    if confidence < 0.5:
        return "INI BUKAN REMPAH SAYANG"
    else:
        return f"Predicted Class: {predicted_class} with Confidence: {confidence:.2f}"

# Membuat antarmuka Gradio
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title="Spice Image Classification",
    description="Unggah gambar rempah untuk mendapatkan hasil klasifikasi."
)

interface.launch()